In [ ]:
import pandas as pd
import numpy as np
import os 
import geopandas as gpd
import plotly.express as px
from scipy.spatial import KDTree

def count_nearby_points_kdtree(df, lat, lon, radius):
    # 위도와 경도를 라디안으로 변환
    df_rad = np.radians(df[['latitude', 'longitude']])
    point_rad = np.radians([lat, lon])
    
    # KDTree 생성
    tree = KDTree(df_rad)
    
    # 주어진 반경 내의 포인트 검색
    indices = tree.query_ball_point(point_rad, np.radians(radius/6371))
    return len(indices)

# (37.6, 127.1) 주변 10km 이내의 포인트 개수 계산
count = count_nearby_points_kdtree(df, 37.6, 127.1, 10)
print(count)


# 바다숲 지형정보

In [7]:
mf = gpd.read_file('./Marine_forest/Marine_forest.shp')
count = len(mf[(mf['면적'] == mf['구역']) & (mf['면적'] == mf['규모'])])
print(count)
mf.drop(columns=['구역','규모'],inplace=True)
col_names = {'시도명': 'province','시군구명':'city','시군구코드':'city_code','위도':'latitude','경도':'longitude','마을어장':'town','면적':'area'}
mf.rename(columns=col_names, inplace=True)
#기본 칼럼인 위도 경도는 오류가 있었음
mf['latitude'] = mf['geometry'].y
mf['longitude'] = mf['geometry'].x

219


In [9]:
title = '2013~2022년 바다숲 조성사업 현황'
buff = mf.loc[mf['시설년도'] <= 2022]
buff = buff.to_crs(epsg=32652)
center = {'lat': 35.8, 'lon': buff['longitude'].median()}
fig = px.scatter_mapbox(buff, lat=buff['latitude'], lon=buff['longitude'], size=buff['area'],
                        color=buff['area'], size_max=50, zoom=5, center=center, title =title)
fig.update_layout(mapbox_style="open-street-map", margin={"r":10,"t":70,"l":20,"b":20},
                  width=1000 , height=800)
fig.show()

In [ ]:
title = f'2018년 해양지형정보 2013~2018바다숲'
buff = mf.loc[mf['시설년도'] <= 2022]
buff = buff.to_crs(epsg=32652)

center = {'lat': 35.8, 'lon': buff['longitude'].median()}
fig = px.scatter_mapbox(buff, lat=buff['latitude'], lon=buff['longitude'], size=buff['area'],
                        color=buff['area'], size_max=50, zoom=5, center=center, title =title)
fig.update_layout(mapbox_style="open-street-map", margin={"r":10,"t":70,"l":20,"b":20},
                  width=1000 , height=800)
fig.show()


## 2010년~ 2020년 해양환경 변화 시각화